# 1. Install dependencies

1. On windows, after installing Anaconda:     

Open the Anaconda command window and run the following commands to install the necessary libraries

pip install rasterio  
pip install geopandas 

If necessary, you can find out more about installing rasterio and geopandas here: 
https://rasterio.readthedocs.io/en/latest/installation.html    
https://geopandas.org/en/stable/getting_started/install.html

2. clone CCA_EU from https://github.com/JingyanYu/LandUseDecisions/blob/9bd4747ca46c199b63d7936d16e259e46afff9ec/A%20data-driven%20framework%20to%20manage%20uncertainty/CCA_EU.py

In [ ]:
import os

In [ ]:
import rasterio as rio
from rasterio.plot import show
from rasterio.mask import mask

In [ ]:
import geopandas as gpd

In [ ]:
import CCA_EU

In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [ ]:
# set global value
nodata_value = -200

# 2. FUA, GHSL data

The GHSL built-up raster data publicly available with identifiers doi: 10.2905/jrc-ghsl-10007    
The functional urban area data publicly available with identifier:10.2905/347F0337-F2DA-4592-87B3-E25975EC2C95
Functional urban area vector data: https://ghsl.jrc.ec.europa.eu/ghs_fua.php

Download the data into a (new) "data" folder and unzip. Then run the cell below to preprocess the data. 

Alternatively, you can skip this step and download preprocessed "pickles" instead:
The processed GeoDataFrame of European functional urban areas & growth - the file countries_gpd.pkl can be downloaded at https://figshare.com/s/245ad30270bd7eb12f41. 
You will also need to download pre-processes raster data found here: https://figshare.com/s/406b4f8506dfdb80e9b6
After downloading the files, copy to a (new) "pickled_data" folder.

In [ ]:
# preprocess the input data 
ghs_built_raster_paths = [r'data\GHS_BUILT_LDS1975_GLOBE_R2018A_54009_250_V2_0\GHS_BUILT_LDS1975_GLOBE_R2018A_54009_250_V2_0.tif',
                          r'data\GHS_BUILT_LDS1990_GLOBE_R2018A_54009_250_V2_0\GHS_BUILT_LDS1990_GLOBE_R2018A_54009_250_V2_0.tif',
                          r'data\GHS_BUILT_LDS2000_GLOBE_R2018A_54009_250_V2_0\GHS_BUILT_LDS2000_GLOBE_R2018A_54009_250_V2_0.tif',
                          r'data\GHS_BUILT_LDS2014_GLOBE_R2018A_54009_250_V2_0\GHS_BUILT_LDS2014_GLOBE_R2018A_54009_250_V2_0.tif']

fua_path = r'data\GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0\GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg'


In [ ]:

# Function to binary reclassify GHSL urban percentage to binary urban, using 20% threshold
def categorize_urban_vec(percent):
    threshold = 20
    out = np.full_like(percent, nodata_value)
    out[(percent >= 0) & (percent < threshold) ] = 0
    out[percent >= threshold] = 1
    return out


# List of countries of interest
countries = ['Austria','CzechRepublic', 'Denmark', 'Estonia', 'Finland', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Luxembourg', 'Norway', 'Poland', 'Portugal', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Turkey']+['France','UnitedKingdom','Netherlands',
             'Belgium','Germany']

# Read the FUA file with global FUA's
fua_gdf = gpd.read_file(fua_path)

# Restrict dataset to countries and columns of interest
countries_gpd = fua_gdf[fua_gdf['Cntry_name'].isin(countries)]
countries_gpd = countries_gpd[['eFUA_name','Cntry_name','FUA_area','UC_area','geometry']]

#create columns - urban units at 1975, 1990, 2000, 2014,and urban growth between 75-90, 90-00, 00-14
def add_raster_to_dataframe(dataframe, raster_path, column_name):
        def process(x):
            shape = x['geometry']
            with rio.open(raster_path) as src:    
                out_img, out_transform = mask(src, [shape], crop=True)
            fua_urban_raster = np.squeeze(categorize_urban_vec(out_img))
            try:
                x[column_name] = np.unique(fua_urban_raster,return_counts=True)[1][2]
            except:
                x[column_name] = np.nan
            try:
                x['Transform'] = out_transform;
            except:
                x['Transform'] = np.nan
            try:
                x[column_name + ' raster'] = fua_urban_raster.copy();
            except:
                x[column_name + ' raster'] = np.nan
            return x
        
        dataframe = dataframe.apply(process, axis=1)
        return dataframe
        
countries_gpd = add_raster_to_dataframe(countries_gpd, ghs_built_raster_paths[0],'1975 urban')
countries_gpd = add_raster_to_dataframe(countries_gpd, ghs_built_raster_paths[1],'1990 urban')
countries_gpd = add_raster_to_dataframe(countries_gpd, ghs_built_raster_paths[2],'2000 urban')
countries_gpd = add_raster_to_dataframe(countries_gpd, ghs_built_raster_paths[3],'2014 urban')

# Remove missing data
countries_gpd.dropna(inplace=True)

# Pre-calculate urban growth quantities
countries_gpd['90-00 UG'] = countries_gpd['2000 urban']-countries_gpd['1990 urban'] 
countries_gpd['00-14 UG'] = countries_gpd['2014 urban']-countries_gpd['2000 urban'] 
countries_gpd['75-90 UG'] = countries_gpd['1990 urban']-countries_gpd['1975 urban'] 

#Keep only the FUAs with urban growth
countries_gpd = countries_gpd[(countries_gpd['75-90 UG']>0)&(countries_gpd['90-00 UG']>0)&(countries_gpd['00-14 UG']>0)]

# putting the rasters in separate lists 
categorized_FUA_1975s = countries_gpd['1975 urban raster'].to_list()
categorized_FUA_1990s = countries_gpd['1990 urban raster'].to_list()
categorized_FUA_2000s = countries_gpd['2000 urban raster'].to_list()
categorized_FUA_2014s = countries_gpd['2014 urban raster'].to_list()

# the affine transforms are identical from year to year, so this can be simplified
transforms = countries_gpd['Transform'].to_list()

# remove rasters from gpd 
countries_gpd = countries_gpd.drop(columns=['1975 urban raster'])
countries_gpd = countries_gpd.drop(columns=['1990 urban raster'])
countries_gpd = countries_gpd.drop(columns=['2000 urban raster'])
countries_gpd = countries_gpd.drop(columns=['2014 urban raster'])
countries_gpd = countries_gpd.drop(columns=['Transform'])



In [ ]:
# Store preprocessed data in pickles
if not os.path.isdir("pickled_data"):
    os.makedirs("pickled_data")
    
with open(r"pickled_data\categorized_FUA_1975s.pickle", 'wb') as f:
    pickle.dump(categorized_FUA_1975s, f)
with open(r"pickled_data\categorized_FUA_1990s.pickle", 'wb') as f:
    pickle.dump(categorized_FUA_1990s, f)
with open(r"pickled_data\categorized_FUA_2000s.pickle", 'wb') as f:
    pickle.dump(categorized_FUA_2000s, f)
with open(r"pickled_data\categorized_FUA_2014s.pickle", 'wb') as f:
    pickle.dump(categorized_FUA_2014s, f)
with open(r"pickled_data\out_transforms1975.pkl", 'wb') as f:
    pickle.dump(transforms, f)
with open(r"pickled_data\out_transforms2014.pkl", 'wb') as f:
    pickle.dump(transforms, f)
countries_gpd.to_pickle(r"pickled_data\countries_gpd.pkl")

In [ ]:
# Read preprocessed data from pickles
countries_gpd = pd.read_pickle(r"pickled_data\countries_gpd.pkl")
with open(r'pickled_data\categorized_FUA_1975s.pickle', 'rb') as handle:
    categorized_FUA_1975s = pickle.load(handle)
with open(r'pickled_data\categorized_FUA_1990s.pickle', 'rb') as handle:
    categorized_FUA_1990s = pickle.load(handle)
with open(r'pickled_data\categorized_FUA_2000s.pickle', 'rb') as handle:
     categorized_FUA_2000s = pickle.load(handle)
with open(r'pickled_data\categorized_FUA_2014s.pickle', 'rb') as handle:
     categorized_FUA_2014s = pickle.load(handle)
with open(r'pickled_data\out_transforms1975.pkl', 'rb') as f:
    transforms = pickle.load(f)

In [ ]:
#Show first few lines of table
countries_gpd.head()

# 3. Use CCA

The parameters by scenarios can be found at: https://doi.org/10.6084/m9.figshare.22194244.v1

After downloading, copy the parameters to a "parameters" folder.

In [ ]:
# Load the parameters for modelling growth using the four spatial development scenarios 

paras_fourscenarios = np.load(r'parameters\paras_fourscenarios.npy',allow_pickle=True)
seeds_fourscenarios = np.load(r'parameters\seeds_fourscenarios.npy',allow_pickle=True)

In [ ]:
# Apply the model

# Choose a scenario:
# 0 - compact; 1 - medium compact; 2 - medium dispersed; 3 - dispersed
# here we choose 3 - dispersed
scenario_no = 3 

# Get a random parameter set for the chosen scenario
idx = np.random.randint(len(paras_fourscenarios[scenario_no]))
paras = paras_fourscenarios[scenario_no][idx]
seed = seeds_fourscenarios[scenario_no][idx]

# Margate(UK) has index 649 in the FUA database
fua_index = 649

# We will similate growth starting from 1975 and for the amount of growth that we 
# historically saw over the 75-90 period
total_growth = int(countries_gpd.iloc[fua_index]['75-90 UG'])
print('Amount of growth: ' + str(total_growth))

# Get the inital urban outline map of Margate from the preprocessed data 
initial_map = categorized_FUA_1975s[fua_index].copy()
rows, cols = initial_map.shape

# for display, replace nodata with 0
display_map = initial_map.copy()
display_map[initial_map == nodata_value] = 0
show(display_map)

# set the increment size (default = 15)
growth_per_step = 15

# apply the model
final_map = CCA_EU.CCA_last_snapshot([paras[0],0,paras[1]],[0,paras[2],paras[3]],
                                  seed=seed,landmap=initial_map.copy(),
                                  rows=rows,cols=cols,urban_num=total_growth,trans_num=growth_per_step)
# Show the outline of Margate
countries_gpd.iloc[fua_index].geometry


In [ ]:
# Visualize change plots

def plot_change(before, after, transform, axis, title):
    changes_colours = ListedColormap(['none','r'])
    cmap = ListedColormap(["none","none", "grey"])
    b = before.copy()
    a = after.copy()
    b[b==-200] = 0
    a[a==-200] = 0
    show(a, ax = axis, transform = transform, cmap = cmap )
    changes = a - b
    show(changes,ax=axis,cmap=changes_colours,transform=transform)
    axis.tick_params(labelbottom=False,labelleft=False,axis=u'both', which=u'both',length=0)
    axis.set_title(title)
    return

after = categorized_FUA_1990s[fua_index]
transform = transforms[fua_index]
fig, axes = plt.subplots(1,2,figsize=(10,4))
plot_change(initial_map, after, transform, axes[0], 'Real Changes')
plot_change(initial_map, final_map, transform, axes[1], 'Simulated Changes')


In [ ]:
# Export final map to "final_map.tif"
# This will use the projection system from ghsl

with rio.Env():
    with rio.open(ghs_built_raster_paths[0]) as src_dataset:
        profile = src_dataset.profile
        profile.update(
            driver = 'GTiff',
            dtype = rio.int16,
            count = 1,
            compress= 'lzw',
            width = cols,
            height = rows,
            transform = transforms[fua_index])
    with rio.open('final_map.tif', 'w', **profile) as dst_dataset:
        dst_dataset.write(final_map.astype(rio.int16), 1)        
    with rio.open('initial_map.tif', 'w', **profile) as dst_dataset:
        dst_dataset.write(initial_map.astype(rio.int16), 1)        
    with rio.open('after_map.tif', 'w', **profile) as dst_dataset:
        dst_dataset.write(after.astype(rio.int16), 1)        
  


In [ ]:
# Export FUA outline to shapefile
single_fua = countries_gpd.iloc[[fua_index],:]
single_fua.to_file("fua_outline.shp")                      

# Appendix
1. Parameters for four scenarios

The original parameters can be found at: https://figshare.com/s/650730f9e6fedc44ac1a   
The parameters by scenarios can be found at: https://doi.org/10.6084/m9.figshare.22194244.v1

In [ ]:
# Load the parameters for urban spatial development scenarios projection
# The parameters can be found at: https://figshare.com/s/650730f9e6fedc44ac1a
paths = [ r'results_EU\pattern_kl\Avignon_m_chain2.npy',r'results_EU\pattern_kl\Poitiers_m_chain2.npy',
         r'results_EU\pattern_kl\Belfast_m_chain2.npy',r'results_EU\pattern_kl\Blackwater_m_chain2.npy',
         r'results_EU\pattern_kl\Enschede_m_chain2.npy',r'results_EU\pattern_kl\Utrecht_m_chain2.npy',
         r'results_EU\pattern_kl\Leuven_m_chain2.npy', r'results_EU\pattern_kl\Mons_m_chain2.npy', 
         r'results_EU\pattern_kl\Mönchengladbach_m_chain2.npy',r'results_EU\pattern_kl\Wuppertal_m_chain2.npy']

chains = [np.load(path,allow_pickle=True) for path in paths]
paras_best20_pattern = [chain[0][chain[2].argsort()[:20]] for chain in chains]
seeds_best20_pattern = [chain[1][chain[2].argsort()[:20]] for chain in chains]


In [ ]:
paths = [ r'results_EU\changes_kl\Avignon_m_chain2.npy',r'results_EU\changes_kl\Poitiers_m_chain2.npy',
         r'results_EU\changes_kl\Belfast_m_chain2.npy',r'results_EU\changes_kl\Blackwater_m_chain2.npy',
         r'results_EU\changes_kl\Enschede_m_chain2.npy',r'results_EU\changes_kl\Utrecht_m_chain2.npy',
         r'results_EU\changes_kl\Leuven_m_chain2.npy',r'results_EU\changes_kl\Mons_m_chain2.npy', 
         r'results_EU\changes_kl\Mönchengladbach_m_chain2.npy',r'results_EU\changes_kl\Wuppertal_m_chain2.npy']

chains = [np.load(path,allow_pickle=True) for path in paths]
paras_best20_change = [chain[0][chain[2].argsort()[:20]] for chain in chains]
seeds_best20_change = [chain[1][chain[2].argsort()[:20]] for chain in chains]

In [ ]:
paras_compact = np.concatenate([paras_best20_pattern[1], paras_best20_pattern[2],paras_best20_change[2]])

In [ ]:
seeds_compact = np.concatenate([seeds_best20_pattern[1], seeds_best20_pattern[2],seeds_best20_change[2]])

In [ ]:
paras_mediumcompact = np.concatenate([paras_best20_pattern[-1], paras_best20_pattern[4],
                                      paras_best20_pattern[0],paras_best20_pattern[3],
                                      paras_best20_change[1],paras_best20_change[4]])

In [ ]:
seeds_mediumcompact = np.concatenate([seeds_best20_pattern[-1], seeds_best20_pattern[4],
                                      seeds_best20_pattern[0],seeds_best20_pattern[3],
                                      seeds_best20_change[1],seeds_best20_change[4]])

In [ ]:
paras_mediumdispersed = np.concatenate([paras_best20_pattern[5], paras_best20_pattern[-2],
                                      paras_best20_pattern[-3],paras_best20_change[0],
                                      paras_best20_change[-1],paras_best20_change[5]])

In [ ]:
seeds_mediumdispersed = np.concatenate([seeds_best20_pattern[5], seeds_best20_pattern[-2],
                                      seeds_best20_pattern[-3],seeds_best20_change[0],
                                      seeds_best20_change[-1],seeds_best20_change[5]])

In [ ]:
paras_dispersed = np.concatenate([paras_best20_pattern[-4], 
                                      paras_best20_change[-4],paras_best20_change[-3],
                                      paras_best20_change[3],paras_best20_change[-2]])

In [ ]:
seeds_dispersed = np.concatenate([seeds_best20_pattern[-4], 
                                      seeds_best20_change[-4],seeds_best20_change[-3],
                                      seeds_best20_change[3],seeds_best20_change[-2]])

In [ ]:
paras_fourscenarios = [paras_compact,paras_mediumcompact,paras_mediumdispersed,paras_dispersed]
seeds_fourscenarios = [seeds_compact,seeds_mediumcompact,seeds_mediumdispersed,seeds_dispersed]

In [ ]:
np.save('paras_fourscenarios.npy',paras_fourscenarios)
np.save('seeds_fourscenarios.npy',seeds_fourscenarios)